In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-138323", subscription_id="f9d5a085-54dc-4215-9ba6-dad5d86e60a0",
                  resource_group="aml-quickstarts-138323")

exp = Experiment(workspace=ws, name="udacity-project-Obinna")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-138323
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-138323


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "project-1"
try:
    compute_cluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
    
compute_cluster.wait_for_completion(show_output=True)

project-1 exists already
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
#can try this with Grid Sampling and Bayesian Sampling as well
ps = RandomParameterSampling({"--C": choice(1,3,5,7), "--max_iter": choice(75,100,125,150,175)})

# Specify a Policy
#slack_factor used to calculate the allowed distance from the best performing run
#delay_evaluation for delay in evaluation_interval
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=3)

#creating a training folder if not already exist
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py 
#using default params of train.py
est = SKLearn(source_directory='./', compute_target=compute_cluster, entry_script='train.py', script_params={
    '--C':1,'--max_iter':100})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#max_total_runs to be updated later if project takes more time than allocated
hyperdrive_config = HyperDriveConfig(estimator=est, policy=policy, primary_metric_name="Accuracy",
                                     hyperparameter_sampling=ps, max_total_runs=20,
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#getting best run and metric via hyper drive lab instructions
run_top = hyperdrive_run.get_best_run_by_primary_metric()
run_top_metric = run_top.get_metrics()

#printing top run id and mertic
print(run_top.id)
print(run_top_metric['Accuracy'])

#saving the model from top run
best_model = run_top.register_model(model_name = 'best_model_acc', model_path = './')
### YOUR CODE HERE ###

HD_aaa0e913-632c-4a12-ab04-1c62222cf464_19
0.9141781985918912


In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(data)

In [15]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = compute_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data= ds,
    label_column_name='y',
    n_cross_validations=5)


In [17]:
# Submit your automl run

### YOUR CODE HERE ###
automl_exp = Experiment(ws, "automl_Obinna")
automl_run = automl_exp.submit(config = automl_config, show_output = True)
RunDetails(automl_run).show()

Running on remote.
No run_configuration provided, running on project-1 with default configuration
Running on remote compute: project-1
Parent Run ID: AutoML_f75d59d2-c943-49a4-a467-2bc5444d1a55

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-----------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [18]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
top_run, top_model = automl_run.get_output()

print(top_run)

top_run.register_model(model_name = 'best_automlmodel_exp2.pkl', model_path = './outputs/')

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automl_Obinna,
Id: AutoML_f75d59d2-c943-49a4-a467-2bc5444d1a55_22,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-138323', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-138323'), name=best_automlmodel_exp2.pkl, id=best_automlmodel_exp2.pkl:1, version=1, tags={}, properties={})

In [19]:
#deleting the compute cluster
compute_cluster.delete()


Current provisioning state of AmlCompute is "Deleting"

